In [15]:
import os
import importlib
import melee 
import numpy as np
import pandas as pd
import data_processing 
importlib.reload(data_processing) 
from data_processing import process_slp_file, save_processed_data, process_and_save_sequences, process_archive_files_streaming, batch_process_archives, process_directory_slp_files

In [16]:
os.getcwd()

'/Users/alexoon/Desktop/smash_bot/icl_smash'

In [17]:
# Process a single file
slp_file = "./data/Stream-Game_20220828T223051.slp"
sampled_data, columns = process_slp_file(slp_file)
sampled_data.shape

# Create and save sequences
input_seqs, target_seqs = process_and_save_sequences(
    sampled_data, 
    "./training_data", 
    "[libmeleeV2]example1",
    input_len=10,
    target_len=5
)
# Save the results
# save_processed_data(sampled_data, columns, "./training_data", "[libmeleeV2]training_ex1.npy", save_csv=True)

# Or use the main function directly
# shape = main(slp_file, "./training_data", "DESIRED_FILENAME")

Saved sequences to ./training_data/[libmeleeV2]example1_sequences.npz
Input shape: (828, 113, 10)
Target shape: (828, 113, 5)


In [18]:
# To load the sequences later:
loaded = np.load("./training_data/[libmeleeV2]example1_sequences.npz", allow_pickle=True)
input_seqs = loaded['inputs']
target_seqs = loaded['targets']

In [19]:
os.getcwd()

'/Users/alexoon/Desktop/smash_bot/icl_smash'

In [20]:
columns

['frame',
 'distance',
 'p3_action_frame',
 'p3_controller_c_stick_x',
 'p3_controller_c_stick_y',
 'p3_controller_l_shoulder',
 'p3_controller_r_shoulder',
 'p3_controller_main_stick_x',
 'p3_controller_main_stick_y',
 'p3_button_button_a',
 'p3_button_button_b',
 'p3_button_button_x',
 'p3_button_button_y',
 'p3_button_button_z',
 'p3_button_button_l',
 'p3_button_button_r',
 'p3_button_button_start',
 'p3_button_button_d_up',
 'p3_button_button_d_down',
 'p3_button_button_d_left',
 'p3_button_button_d_right',
 'p3_ecb_bottom_x',
 'p3_ecb_bottom_y',
 'p3_ecb_left_x',
 'p3_ecb_left_y',
 'p3_ecb_right_x',
 'p3_ecb_right_y',
 'p3_ecb_top_x',
 'p3_ecb_top_y',
 'p3_facing',
 'p3_hitlag_left',
 'p3_hitstun_frames_left',
 'p3_iasa',
 'p3_invulnerability_left',
 'p3_invulnerable',
 'p3_jumps_left',
 'p3_moonwalkwarning',
 'p3_nana',
 'p3_off_stage',
 'p3_on_ground',
 'p3_percent',
 'p3_position_x',
 'p3_position_y',
 'p3_shield_strength',
 'p3_speed_air_x_self',
 'p3_speed_ground_x_self',
 '

In [ ]:
df = pd.read_csv("/Users/alexoon/Desktop/smash_bot/icl_smash/old_data/[libmeleeV2]training_ex1.npy_data.csv")
df.head() 
# Order of ENUMS: stage, p3_action, p3_character, p4_action, p4_character (could replace p3 and p4 with p1 and p2 respectively)

,frame,distance,p3_action_frame,p3_controller_c_stick_x,p3_controller_c_stick_y,p3_controller_l_shoulder,p3_controller_r_shoulder,p3_controller_main_stick_x,p3_controller_main_stick_y,p3_button_button_a,...,proj1_owner,proj1_position_x,proj1_position_y,proj1_speed_x,proj1_speed_y,stage,p3_action,p3_character,p4_action,p4_character
0,-123,94.000052,-1,0.5,0.5,0.0,0.0,0.5,0.5,1,...,NaN,NaN,NaN,NaN,NaN,DREAMLAND,ENTRY,SHEIK,ENTRY,JIGGLYPUFF
1,-108,94.000021,1,0.5,0.5,0.0,0.0,0.5,0.5,1,...,NaN,NaN,NaN,NaN,NaN,DREAMLAND,ENTRY_START,SHEIK,ENTRY,JIGGLYPUFF
2,-93,94.000026,11,0.5,0.5,0.0,0.0,0.5,0.5,1,...,NaN,NaN,NaN,NaN,NaN,DREAMLAND,ENTRY_START,SHEIK,ENTRY_START,JIGGLYPUFF
3,-78,94.000010,-1,0.5,0.5,0.0,0.0,0.5,0.5,1,...,NaN,NaN,NaN,NaN,NaN,DREAMLAND,ENTRY_END,SHEIK,ENTRY_START,JIGGLYPUFF
4,-63,94.000639,-1,0.5,0.5,0.0,0.0,0.5,0.5,1,...,NaN,NaN,NaN,NaN,NaN,DREAMLAND,ENTRY_END,SHEIK,ENTRY_END,JIGGLYPUFF


In [ ]:
# batch_process_archives("./old_data/", "./training_data/")
process_archive_files_streaming("./old_data/example_7z.7z", "./training_data")

Processing 7Z files:  33%|███▎      | 2/6 [00:03<00:06,  1.57s/it]

Saved sequences to ./training_data/BTSSmash-Game_20200126T202356_sequences.npz
Input shape: (1572, 10, 113)
Target shape: (1572, 5, 113)
❌ Failed to process data/Day 3-Game_20210718T094500.slp: 61 is not a valid EventType


Processing 7Z files:  50%|█████     | 3/6 [00:03<00:02,  1.05it/s]

Saved sequences to ./training_data/Game_001AE992EB29_20210714T205828_sequences.npz
Input shape: (19, 10, 211)
Target shape: (19, 5, 211)


Processing 7Z files:  67%|██████▋   | 4/6 [00:05<00:02,  1.33s/it]

Saved sequences to ./training_data/Stream-Game_20220828T223051_sequences.npz
Input shape: (827, 10, 113)
Target shape: (827, 5, 113)


Processing 7Z files:  83%|████████▎ | 5/6 [00:07<00:01,  1.30s/it]

Saved sequences to ./training_data/Stream-Game_20220828T225101_sequences.npz
Input shape: (525, 10, 113)
Target shape: (525, 5, 113)


Processing 7Z files: 100%|██████████| 6/6 [00:09<00:00,  1.53s/it]

Saved sequences to ./training_data/Stream-Game_20220828T225335_sequences.npz
Input shape: (796, 10, 113)
Target shape: (796, 5, 113)

Processed 5 files from old_data/example_7z.7z
Failed to process 1 files
Failed files:
  - data/Day 3-Game_20210718T094500.slp
Results saved to: ./training_data


In [14]:
# Define input and output directories
data_dir = "./data"
output_dir = "./training_data"

# Make sure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Process all .slp files
processed_count, failed = process_directory_slp_files(
    directory_path=data_dir,
    output_dir=output_dir,
    input_len=10,
    target_len=5
)

Found 6 .slp files to process


Processing .slp files:  33%|███▎      | 2/6 [00:02<00:04,  1.15s/it]

Saved sequences to ./training_data/Stream-Game_20220828T225101_sequences.npz
Input shape: (526, 113, 10)
Target shape: (526, 113, 5)


Processing .slp files:  50%|█████     | 3/6 [00:02<00:02,  1.23it/s]

Saved sequences to ./training_data/Game_001AE992EB29_20210714T205828_sequences.npz
Input shape: (20, 211, 10)
Target shape: (20, 211, 5)


...

Saved sequences to ./training_data/BTSSmash-Game_20200126T202356_sequences.npz
Input shape: (1573, 113, 10)
Target shape: (1573, 113, 5)


Processing .slp files:  83%|████████▎ | 5/6 [00:11<00:02,  2.73s/it]

Saved sequences to ./training_data/Stream-Game_20220828T223051_sequences.npz
Input shape: (828, 113, 10)
Target shape: (828, 113, 5)


Processing .slp files: 100%|██████████| 6/6 [00:13<00:00,  2.31s/it]

Saved sequences to ./training_data/Stream-Game_20220828T225335_sequences.npz
Input shape: (797, 113, 10)
Target shape: (797, 113, 5)

Successfully processed 5 files
Failed to process 1 files:
  - data/Day 3-Game_20210718T094500.slp: 61 is not a valid EventType
Results saved to: ./training_data
